In [30]:
import pandas as pd
import psycopg2

In [31]:
# Load the raw sales data into a pandas DataFrame
data = pd.read_csv("./data/superstore_dataset2011-2015.csv", encoding="latin1")

In [32]:
print(data.head())

   Row ID         Order ID Order Date Ship Date       Ship Mode Customer ID  \
0   42433     AG-2011-2040   1/1/2011  6/1/2011  Standard Class    TB-11280   
1   22253    IN-2011-47883   1/1/2011  8/1/2011  Standard Class    JH-15985   
2   48883     HU-2011-1220   1/1/2011  5/1/2011    Second Class      AT-735   
3   11731  IT-2011-3647632   1/1/2011  5/1/2011    Second Class    EM-14140   
4   22255    IN-2011-47883   1/1/2011  8/1/2011  Standard Class    JH-15985   

     Customer Name      Segment         City            State  ...  \
0  Toby Braunhardt     Consumer  Constantine      Constantine  ...   
1      Joseph Holt     Consumer  Wagga Wagga  New South Wales  ...   
2    Annie Thurman     Consumer     Budapest         Budapest  ...   
3     Eugene Moren  Home Office    Stockholm        Stockholm  ...   
4      Joseph Holt     Consumer  Wagga Wagga  New South Wales  ...   

         Product ID         Category Sub-Category  \
0  OFF-TEN-10000025  Office Supplies      Storage  

In [33]:
# Data Transformation
# Example transformations: You can modify these based on your preferences

# Convert the Order Date column to datetime format
data['Order Date'] = pd.to_datetime(data['Order Date'])

c:\Users\Nirosh.NIROSHSR\miniconda3\envs\mlenv\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2011' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\Nirosh.NIROSHSR\miniconda3\envs\mlenv\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2012' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\Nirosh.NIROSHSR\miniconda3\envs\mlenv\lib\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13-01-2013' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
c:\Users\Nirosh.NIROSHSR\miniconda3\envs\mlenv\lib\site-packages\pandas\core\tools\datetimes.p

In [34]:
# Extract the month and year from the Order Date column
data['Month'] = data['Order Date'].dt.month
data['Year'] = data['Order Date'].dt.year

In [35]:
# Group by month and year, and calculate the total sales
monthly_sales = data.groupby(['Month', 'Year', 'Product Name'])['Sales'].sum().reset_index()

In [36]:
print(monthly_sales.head())

   Month  Year                      Product Name    Sales
0      1  2011  3M Hangers With Command Adhesive   22.200
1      1  2011      4009 Highlighters by Sanford   19.900
2      1  2011   9-3/4 Diameter Round Wall Clock   44.128
3      1  2011                 AT&T 841000 Phone  124.200
4      1  2011       Acco 3-Hole Punch, Recycled   30.480


In [37]:
# Find the most popular sales in terms of month and year
most_popular_sales = monthly_sales.sort_values('Sales', ascending=False).head(1)

In [38]:
print(most_popular_sales)

      Month  Year                                       Product Name     Sales
5197      3  2011  Cisco TelePresence System EX90 Videoconferenci...  22638.48


In [39]:
# Group by month, year, and product name, and calculate the total sales
category_sales = data.groupby('Category')['Sales'].sum().reset_index()

In [40]:
print(category_sales.head())

          Category         Sales
0        Furniture  4.110874e+06
1  Office Supplies  3.787070e+06
2       Technology  4.744557e+06


In [41]:
# Find the top-selling category overall
top_category_sales = category_sales.sort_values('Sales', ascending=False).head(1)

In [42]:
print(category_sales.head())

          Category         Sales
0        Furniture  4.110874e+06
1  Office Supplies  3.787070e+06
2       Technology  4.744557e+06


In [43]:
# Connect to PostgreSQL database
conn = psycopg2.connect(
    host="containers-us-west-190.railway.app",
    database="railway",
    user="postgres",
    password="u1npW6SnjJz3BHDi2Whx",
    port="7259"
)

In [44]:
# Create a cursor to execute SQL queries
cur = conn.cursor()

In [45]:
# Create the tables in the database (if they don't exist)
cur.execute("""
    CREATE TABLE monthly_sales (
        month INT,
        year INT,
         product_name TEXT,
        sales FLOAT
    )
""")

cur.execute("""
    CREATE TABLE top_category_sales (
        category TEXT,
        sales FLOAT
    )
""")

In [46]:
# Convert the transformed data to a list of tuples for insertion
monthly_sales_values = [tuple(row) for row in monthly_sales.values]


top_category_sales_values = [tuple(row) for row in top_category_sales.itertuples(index=False)]

In [48]:

# Execute the SQL insert statements to save the transformed data into the database table
cur.executemany("""
    INSERT INTO monthly_sales (month, year, product_name, sales)
    VALUES (%s, %s, %s, %s)
""", 
# limit to 20 rows for testing
monthly_sales_values[:20]
)

cur.executemany("""
    INSERT INTO top_category_sales (category, sales)
    VALUES (%s, %s)
""", top_category_sales_values)

In [49]:
# Commit the changes to the database
conn.commit()

# Close the database connection
cur.close()
conn.close()